In [1]:
import tensorflow as tf
import urllib

keras_model = tf.keras.applications.MobileNetV2(
    weights="imagenet",
    input_shape=(224, 224, 3),
    classes=1000,
)

label_url = "https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt"
class_labels = urllib.request.urlopen(label_url).read().splitlines()
class_labels = class_labels[1:]

assert len(class_labels) == 1000

for i, label in enumerate(class_labels):
    if isinstance(label, bytes):
        class_labels[i] = label.decode("utf-8")

14536120/14536120 [==============================] - 2s 0us/step


In [2]:
import coremltools as ct 

image_input = ct.ImageType(
    shape = (1, 224, 224, 3),
    bias = [-1, -1, -1],
    scale=1/127
)

classifier_config = ct.ClassifierConfig(class_labels)

model = ct.convert(
    keras_model,
    inputs = [image_input],
    classifier_config = classifier_config
)




TensorFlow version 2.14.0 has not been tested with coremltools. You may run into unexpected errors. TensorFlow 2.12.0 is the most recent version that has been tested.
Torch version 2.0.1 has not been tested with coremltools. You may run into unexpected errors. Torch 2.0.0 is the most recent version that has been tested.
When both 'convert_to' and 'minimum_deployment_target' not specified, 'convert_to' is set to "mlprogram" and 'minimum_deployment_targer' is set to ct.target.iOS15 (which is same as ct.target.macOS12). Note: the model will not run on systems older than iOS15/macOS12/watchOS8/tvOS15. In order to make your model run on older system, please set the 'minimum_deployment_target' to iOS14/iOS13. Details please see the link: https://coremltools.readme.io/docs/unified-conversion-api#target-conversion-formats
Running MIL backend_mlprogram pipeline: 100%|██████████| 12/12 [00:00<00:00, 1082.08 passes/s]


In [3]:
print(model.get_spec().description)

input {
  name: "input_1"
  type {
    imageType {
      width: 224
      height: 224
      colorSpace: RGB
    }
  }
}
output {
  name: "classLabel"
  type {
    stringType {
    }
  }
}
output {
  name: "classLabel_probs"
  type {
    dictionaryType {
      stringKeyType {
      }
    }
  }
}
predictedFeatureName: "classLabel"
predictedProbabilitiesName: "classLabel_probs"
metadata {
  userDefined {
    key: "com.github.apple.coremltools.source"
    value: "tensorflow==2.14.0"
  }
  userDefined {
    key: "com.github.apple.coremltools.version"
    value: "7.0"
  }
}



In [6]:
model.input_description["input_1"] = "입력 이미지"
model.output_description["classLabel"] = "이미지 카테고리"

model.author = "ykoon"
model.license = "Apache2.0"

model.short_description = "my new mlops model"
model.version = "2.0"

model.save("ykmodel")